In [1]:
import pandas as pd
import os
import requests
from datetime import datetime

os.chdir("../../")

from src import display_df

In [2]:
from datetime import date
import holidays

In [3]:
df = pd.read_csv('data/clean/processed_data.csv')

In [4]:
# df = df.head(20000)
# # save to cvs
# df.to_csv('data/clean/processed_data_sample.csv', index=False)

#### Using holidays library

In [5]:
ng_holidays = holidays.Nigeria(years=pd.to_datetime(df['trip_start_date']).dt.year.unique())

In [6]:
ng_holidays

{datetime.date(2021, 1, 1): "New Year's Day", datetime.date(2021, 4, 2): 'Good Friday', datetime.date(2021, 4, 5): 'Easter Monday', datetime.date(2021, 5, 1): "Workers' Day", datetime.date(2021, 6, 12): 'Democracy Day', datetime.date(2021, 10, 1): 'Independence Day', datetime.date(2021, 12, 25): 'Christmas Day', datetime.date(2021, 12, 26): 'Boxing Day', datetime.date(2021, 5, 13): 'Eid-el-Fitr (estimated)', datetime.date(2021, 5, 14): 'Eid-el-Fitr Holiday (estimated)', datetime.date(2021, 7, 20): 'Eid-el-Kabir (estimated)', datetime.date(2021, 7, 21): 'Eid-el-Kabir Holiday (estimated)', datetime.date(2021, 10, 18): 'Eid-el-Mawlid (estimated)', datetime.date(2021, 5, 3): "Workers' Day (observed)", datetime.date(2021, 6, 14): 'Democracy Day (observed)', datetime.date(2021, 12, 27): 'Christmas Day (observed)', datetime.date(2021, 12, 28): 'Boxing Day (observed)', datetime.date(2022, 1, 1): "New Year's Day", datetime.date(2022, 4, 15): 'Good Friday', datetime.date(2022, 4, 18): 'Easter Mo

In [7]:
# 2. Weekend vs. Weekday
df['is_weekend'] = pd.to_datetime(df['trip_start_time']).dt.dayofweek.isin([5, 6]).astype(int)

In [8]:
def add_holiday_feature(df):
    """Adds a feature 'is_holiday' to the DataFrame."""
    # Create a list of holidays in Nigeria
    Nigeria = holidays.Nigeria(years=pd.to_datetime(df['trip_start_date']).dt.year.unique())

    # Add missing holidays
    Nigeria[datetime(2021, 6, 12)] = "Democracy Day"
    Nigeria[datetime(2021, 8, 9)] = "Islamic New Year"
    Nigeria[datetime(2021, 10, 15)] = "Eid al-Fitr"
    
    # Create a new column 'is_holiday'
    df['trip_start_date'] = pd.to_datetime(df['trip_start_date']).dt.date

    df['is_holiday'] = df['trip_start_date'].apply(lambda date: date in Nigeria).astype(int)

    return df

In [9]:
new_df = add_holiday_feature(df)

In [10]:
# display unique values of is_holiday using df inbuilt function
new_df['is_holiday'].value_counts()

0    1557740
Name: is_holiday, dtype: int64

In [11]:
df.to_csv('data/features/features_data.csv', index=False)

#### Using date nager api

In [6]:
# Get public holidays for Nigeria (replace with appropriate API/data source if needed)
def get_public_holidays(year):
    url = f"https://date.nager.at/api/v3/PublicHolidays/{year}/NG"
    response = requests.get(url)
    if response.status_code == 200:
        return [datetime.strptime(holiday['date'], '%Y-%m-%d').date() for holiday in response.json()]
    else:
        return []

holidays_2021 = get_public_holidays(2021)

df['is_holiday'] = df['trip_start_date'].isin(holidays_2021).astype(int)

# 2. Weekend vs. Weekday
df['is_weekend'] = pd.to_datetime(df['trip_start_time']).dt.dayofweek.isin([5, 6]).astype(int)

In [7]:
ng_holidays = holidays.NG(years=[2021])

In [11]:
print(ng_holidays)

{datetime.date(2021, 1, 1): "New Year's Day", datetime.date(2021, 4, 2): 'Good Friday', datetime.date(2021, 4, 5): 'Easter Monday', datetime.date(2021, 5, 1): "Workers' Day", datetime.date(2021, 6, 12): 'Democracy Day', datetime.date(2021, 10, 1): 'Independence Day', datetime.date(2021, 12, 25): 'Christmas Day', datetime.date(2021, 12, 26): 'Boxing Day', datetime.date(2021, 5, 13): 'Eid-el-Fitr (estimated)', datetime.date(2021, 5, 14): 'Eid-el-Fitr Holiday (estimated)', datetime.date(2021, 7, 20): 'Eid-el-Kabir (estimated)', datetime.date(2021, 7, 21): 'Eid-el-Kabir Holiday (estimated)', datetime.date(2021, 10, 18): 'Eid-el-Mawlid (estimated)', datetime.date(2021, 5, 3): "Workers' Day (observed)", datetime.date(2021, 6, 14): 'Democracy Day (observed)', datetime.date(2021, 12, 27): 'Christmas Day (observed)', datetime.date(2021, 12, 28): 'Boxing Day (observed)'}
